# STFT Denoising
This is a jupyter notebook for processing of waveform data in the time-domain into STFT data.

In [ ]:
import os
import numpy as np

import matplotlib.pyplot as plt

from scipy.signal import stft, istft


from ipywidgets import interact